## MOONRAKER HACK 6124
## Speach Fact Checker
### Using LangGraph - Agent Chain with multiple tools

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

### Dependencies
. . .

In [1]:
!pip install -qU langchain langchain_openai langgraph arxiv duckduckgo-search tavily-python

### Environment Variables
OpenAI API, Tavily and LangSmith environment variables.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE2 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

### Agent Tool Belt

Adding tools that equip our agent with a toolbelt to help answer questions and add external knowledge.

In [5]:
#from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.tavily_search import TavilySearchResults

tool_belt = [
#    DuckDuckGoSearchRun(),
    TavilySearchResults(), # TavilyRun() replaced duckduckgo-search because of rate limiting
    ArxivQueryRun() # Maybe use a different tool here for sentiment and all
]

### Actioning with Tools
The Agent will use a ToolExecutor 

In [6]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tool_belt)

### Model

Using OpenAI LLM, leveraging the OpenAI function calling API.

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

OK, let's "put on the tool belt"

In [8]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

### The Agent State Machine
LangGraph StatefulGraph - AgentState object.

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [9]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

### Build the Graph

In [10]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = tool_executor.invoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

Now we have two total nodes. We have:
- `call_model` is a node that will...well...call the model
- `call_tool` is a node which will call a tool

In [11]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [12]:
workflow.set_entry_point("agent")

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow. We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

In [13]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [15]:
workflow.add_edge("action", "agent")

All that's left to do now is to compile our workflow - and we're off!

In [16]:
app = workflow.compile()

#### Helper Function to print messages

In [17]:
def print_messages(messages):
  next_is_tool = False
  initial_query = True
  for message in messages["messages"]:
    if "function_call" in message.additional_kwargs:
      print()
      print(f'Tool Call - Name: {message.additional_kwargs["function_call"]["name"]} + Query: {message.additional_kwargs["function_call"]["arguments"]}')
      next_is_tool = True
      continue
    if next_is_tool:
      print(f"Tool Response: {message.content}")
      next_is_tool = False
      continue
    if initial_query:
      print(f"Initial Query: {message.content}")
      print()
      initial_query = False
      continue
    print()
    print(f"Agent Response: {message.content}")


## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!


In [18]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="What is RAG in the context of Large Language Models? When did it break onto the scene?")]}
#inputs = {"messages" : [HumanMessage(content="What is RAG?")]}
messages = app.invoke(inputs)

print_messages(messages)

Initial Query: What is RAG in the context of Large Language Models? When did it break onto the scene?


Agent Response: RAG stands for Retrieval-Augmented Generation. It is a technique used in Large Language Models to improve the generation of text by combining retrieval-based methods with generative models. RAG models use a retriever to find relevant passages of text from a large corpus of documents and then generate text based on the retrieved information.

RAG broke onto the scene in 2020 when a research paper titled "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks" was published by researchers from Facebook AI. This paper introduced the RAG framework and demonstrated its effectiveness in various natural language processing tasks. Since then, RAG has gained popularity in the field of NLP for its ability to generate more informative and coherent text by leveraging external knowledge sources.


1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "function_call" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "function_call" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [19]:
inputs = {"messages" : [HumanMessage(content="What is QLoRA in Machine Learning? Are their any technical papers that could help me understand? Once you have that information, can you look up the bio of the first author on the QLoRA paper?")]}

messages = app.invoke(inputs)

print_messages(messages)

Initial Query: What is QLoRA in Machine Learning? Are their any technical papers that could help me understand? Once you have that information, can you look up the bio of the first author on the QLoRA paper?


Tool Call - Name: arxiv + Query: {"query":"QLoRA in Machine Learning"}
Tool Response: Published: 2023-05-23
Title: QLoRA: Efficient Finetuning of Quantized LLMs
Authors: Tim Dettmers, Artidoro Pagnoni, Ari Holtzman, Luke Zettlemoyer
Summary: We present QLoRA, an efficient finetuning approach that reduces memory usage
enough to finetune a 65B parameter model on a single 48GB GPU while preserving
full 16-bit finetuning task performance. QLoRA backpropagates gradients through
a frozen, 4-bit quantized pretrained language model into Low Rank
Adapters~(LoRA). Our best model family, which we name Guanaco, outperforms all
previous openly released models on the Vicuna benchmark, reaching 99.3% of the
performance level of ChatGPT while only requiring 24 hours of finetuning on a
single GPU

### EXTRA 
### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [20]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | app | parse_output

In [21]:
agent_chain.invoke({"question" : "What is RAG for LLM Applications"})

'RAG (Retrieval Augmented Generation) for LLM (Large Language Models) applications allows us to give foundational models local context without expensive fine-tuning. It can be done on normal everyday machines like laptops. If you are interested in getting started with RAG for LLM-powered applications, you can refer to tutorials and guides available online:\n\n1. [LLM RAG Tutorial](https://colab.research.google.com/github/SamHollings/llm_tutorial/blob/main/llm_tutorial_rag.ipynb): This tutorial provides a simple introduction to getting started with an LLM to create a RAG app.\n\n2. [Your Guide to Starting With RAG for LLM-Powered Applications](https://medium.com/@caldhubaib/your-guide-to-starting-with-rag-for-llm-powered-applications-ee5ce31cab71): This guide offers insights and tips on building enterprise-grade LLMs and getting started with RAG for LLMs.\n\n3. [Basic Conversational AI with RAG Solutions](https://github.com/zahaby/intro-llm-rag): This guide is for technical teams intere